### Requiere HTTPS

En este ejercicio crearemos una Política de S3 Bucket que requiere que las conexiones utilicen HTTPS.

In [48]:
import boto3
import json
import random
from pprint import pprint
from botocore.exceptions import ClientError

region_aws = 'us-east-1'
s3 = boto3.resource('s3', region_name=region_aws)
s3_client = boto3.client('s3', region_name=region_aws)
transfer = boto3.s3.transfer.S3Transfer(client=s3_client)

In [38]:
bucket = 'cookbook-bucket-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket)
print(f"Bucket: {bucket}")

Bucket: s3.Bucket(name='cookbook-bucket-68492')


In [39]:
policy = {
    "Statement": [
    {
    "Action": "s3:*",
    "Effect": "Deny",
    "Principal": "*",
    "Resource": f"arn:aws:s3:::{bucket.name}/*",
    "Condition": {
        "Bool": {
            "aws:SecureTransport": False
            }
        }
        }
    ]
}

bucket.Policy().put(Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '61NMT9QJZXADADW9',
  'HostId': 'xOcNoDs0GUiAM/PGN9kbmL+shlHswLZhr5+PVMoWsWFjNMjseoRBWqxIxTn3n8tQTR/CGYS1hH4=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'xOcNoDs0GUiAM/PGN9kbmL+shlHswLZhr5+PVMoWsWFjNMjseoRBWqxIxTn3n8tQTR/CGYS1hH4=',
   'x-amz-request-id': '61NMT9QJZXADADW9',
   'date': 'Mon, 31 Jul 2023 13:41:32 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

#### Test http

In [46]:
s3_http = boto3.client('s3', endpoint_url='http://s3.amazonaws.com', region_name=region_aws)

# Upload a sample file
key_name = 'ip-ranges.json'
bucket.upload_file(key_name, key_name)

try:
    response = s3_http.head_object(
        Bucket=bucket.name,
        Key=key_name
    )
    pprint(response)
except ClientError as e:
    print(f"Error: {e}")

Error: An error occurred (403) when calling the HeadObject operation: Forbidden


* El comando debería devolver un error 403 ya que el endpoint-url es HTTP.

#### Test https

In [49]:
s3_https = boto3.client('s3', endpoint_url='https://s3.amazonaws.com', region_name=region_aws)

# Upload a sample file
key_name = 'ip-ranges.json'
bucket.upload_file(key_name, key_name)

try:
    response = s3_https.head_object(
        Bucket=bucket.name,
        Key=key_name
    )
    pprint(response)
except ClientError as e:
    print(f"Error: {e}")

{'AcceptRanges': 'bytes',
 'ContentLength': 1348061,
 'ContentType': 'binary/octet-stream',
 'ETag': '"a4c4f220536a3a2bbfb53e6772039d1d"',
 'LastModified': datetime.datetime(2023, 7, 31, 13, 52, 11, tzinfo=tzutc()),
 'Metadata': {},
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
                                      'content-length': '1348061',
                                      'content-type': 'binary/octet-stream',
                                      'date': 'Mon, 31 Jul 2023 13:52:13 GMT',
                                      'etag': '"a4c4f220536a3a2bbfb53e6772039d1d"',
                                      'last-modified': 'Mon, 31 Jul 2023 '
                                                       '13:52:11 GMT',
                                      'server': 'AmazonS3',
                                      'x-amz-id-2': 'F1dwPvJf1qIZUr3yGro0eLuTVpYdW8KlHSi9p2ac0mftUoDA61XngGWOsKIjR+KCN7c+q6mZ0Hg=',
                                      'x-amz-request-id': 

### Require SSE-S3 Encryption

En este ejercicio vamos a crear una Política de S3 Bucket que requiere cifrado de datos en reposo.

In [52]:
policy = {
    "Statement": [
        {
            "Effect": "Deny",
            "Principal": "*",
            "Action": "s3:PutObject",
            "Resource": f"arn:aws:s3:::{bucket.name}/*",
            "Condition": {
                "StringNotEquals": {
                    "s3:x-amz-server-side-encryption": "AES256"
                }
            }
        }
    ]
}

bucket.Policy().put(Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '1FEB21CS86AVZKK5',
  'HostId': 'gqkC3STMi/jNcz1zi3lw6279bXd+8rL1/8kh5H9mvBBlQd8VLMxcV5kLbWNoz5JkVnAJ5uYtakY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'gqkC3STMi/jNcz1zi3lw6279bXd+8rL1/8kh5H9mvBBlQd8VLMxcV5kLbWNoz5JkVnAJ5uYtakY=',
   'x-amz-request-id': '1FEB21CS86AVZKK5',
   'date': 'Mon, 31 Jul 2023 13:56:23 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

#### Sin encriptación SSE-S3

In [66]:
sample_file_content = 'This is a sample file content to illustrate the use of S3 Policy Security'
sample_file_name = 'sample-file.txt'
with open(sample_file_name, 'w') as f:
    f.write(sample_file_content)

try:
    transfer.upload_file(sample_file_name,
                     bucket.name, 
                     sample_file_name
    )
except ClientError as e:
    print(f'Error uploading file {sample_file_name} to S3 bucket {bucket.name}: {e}')
except Exception as e:
    print(e) 

Failed to upload sample-file.txt to cookbook-bucket-68492/sample-file.txt: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied


* La petición debería fallar, ya que el objeto no está encriptado usando encriptación SSE-S3 como requiere su política de buckets.

#### Con encriptación SSE-S3

In [67]:
try:
    transfer.upload_file(sample_file_name,
                     bucket.name, 
                     sample_file_name,
                     extra_args={'ServerSideEncryption':'AES256'}
    )
except ClientError as e:
    print(f'Error uploading file {sample_file_name} to S3 bucket {bucket.name}: {e}')
except Exception as e:
    print(e) 

* El comando tuvo éxito porque el PUT utilizó SSE-S3.